In [1]:
# libraries
import pandas as pd 
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno

# Silence warnings 
import warnings
warnings.filterwarnings("ignore")

### Load the data files 

In [2]:
# load the data files 
us_video = pd.read_csv("US_youtube_trending_data.csv")

In [3]:
# load the category id json files
us_category = pd.read_json("US_category_id.json")

### Create a dataframe to match category id and title from the json files

In [4]:
# drop the unnecessary column in the category data set 
us_category = us_category['items']

In [5]:
# create an empty dataframe for matching id and category title 
id_title = pd.DataFrame()

# create an empty list for us category id 
id_us = []

# create an empty list for us category title 
title_us = []

# append the id extracted from the us category to the list 
for i in range(len(us_category)):
    id_us.append(us_category[i]['id'])

# append the title extracted from the us category to the list 
for i in range(len(us_category)):
    title_us.append(us_category[i]['snippet']['title'])
    
# assign the lists to columns in the id_title dataframe 
id_title['category_id'] = id_us
id_title['category_title'] = title_us

### Merge the category with the video data file 

In [6]:
# change the column name in the video dataframe 
us_video = us_video.rename(columns = {'categoryId':'category_id'})

# change the data type for merging
id_title['category_id'] = id_title['category_id'].astype("int")

In [7]:
# merge the id_title with video dataframe
us_video = us_video.merge(id_title, how = 'inner', on = 'category_id')

In [9]:
# drop unnecessary columns and rearrange the column order 
us_video = us_video[['video_id','category_id','category_title','publishedAt','trending_date','view_count','likes','dislikes','comment_count','comments_disabled','ratings_disabled']]

### Convert to datetime object 

In [11]:
# create an empty list for trending_date and publish date
trending = []
publish = []

# change to datetime object 
for i in range(len(us_video['trending_date'])):
    temp_trend = us_video['trending_date'][i].split('T')
    trend = datetime.strptime(temp_trend[0],'%Y-%m-%d').date()
    trending.append(trend)

for i in range(len(us_video['publishedAt'])):
    temp_publish = us_video['publishedAt'][i].split('T')
    publish1 = datetime.strptime(temp_publish[0],'%Y-%m-%d').date()
    publish.append(publish1)
    
# assign the datetime object back to the dataframe 
us_video['trending_date'] = trending
us_video['publishedAt'] = publish

### Create a data set for information during the trending period 

In [12]:
# extract the first and last trending date 
trending_days = us_video.groupby('video_id')['trending_date'].agg(['max','min']).reset_index()

# count the trending days 
trending_days['trending_days'] = trending_days['max'] - trending_days['min']

In [13]:
# extract the max number of views and the least number of views 
view_increase = us_video.groupby('video_id')['view_count'].agg(['max','min']).reset_index()

# count the increase in views 
view_increase['view_increase'] = view_increase['max'] - view_increase['min']

# extract the max number of likes and the least number of likes
like_increase = us_video.groupby('video_id')['likes'].agg(['max','min']).reset_index()

# count the increase in likes
like_increase['like_increase'] = like_increase['max'] - like_increase['min']

# extract the max number of comments and the least number of comments
comment_increase = us_video.groupby('video_id')['comment_count'].agg(['max','min']).reset_index()

# count the increase in comments 
comment_increase['comment_increase'] = comment_increase['max'] - comment_increase['min']


In [14]:
# merge the data frames to compose a data frame for trending period information 
trending_period = trending_days.merge(view_increase, how = 'inner', on = 'video_id')
trending_period = trending_period.merge(like_increase, how = 'inner', on = 'video_id')
trending_period = trending_period.merge(comment_increase, how = 'inner', on = 'video_id')
trending_period = trending_period[['video_id','trending_days','view_increase','like_increase','comment_increase']]

In [15]:
# look at sample
trending_period.sample(10)

,video_id,trending_days,view_increase,like_increase,comment_increase
3001,gFYAXsa7pe8,5 days,6807884,419798,23746
874,B4i3JsUr45c,2 days,1288059,37389,946
918,Bhg-Gw953b0,5 days,22992452,451835,255553
2847,e5vv4IUw0pU,5 days,4443681,48137,3463
1715,NkE0AMGzpJY,7 days,14436247,498092,34786
3735,qtOo192Du9Q,6 days,2373635,52797,2185
134,17-ggJi5Lr0,7 days,1874533,42726,2471
2634,axth9vCCx_w,3 days,33880,194,70
2829,dhSgWPDZKDg,5 days,726579,40995,1413
4166,xiFO4SWi40A,5 days,288544,9535,433


### Merge the trending period info with the total dataset

In [17]:
# merge
total = us_video.merge(trending_period, how = 'inner', on = 'video_id')

In [18]:
# look at a sample 
total.sample(10)

,video_id,category_id,category_title,publishedAt,trending_date,view_count,likes,dislikes,comment_count,comments_disabled,ratings_disabled,trending_days,view_increase,like_increase,comment_increase
14885,GViYgk9FXZc,23,Comedy,2020-09-06,2020-09-10,1426440,35917,2565,5131,False,False,6 days,1138709,14456,1839
2746,l4pX35OW1S4,20,Gaming,2020-09-29,2020-10-03,222411,11037,200,1146,False,False,4 days,38263,769,53
10156,QqD46BAe19E,10,Music,2020-09-22,2020-09-28,1064134,64007,1241,3755,False,False,3 days,195822,8722,281
11677,Lj4-SIa9bbk,10,Music,2020-10-30,2020-11-03,3702254,285315,1926,13680,False,False,5 days,2893179,97356,3206
17694,PenhUd-qEDA,1,Film & Animation,2020-11-13,2020-11-19,1115740,100355,1622,6902,False,False,5 days,301993,12997,928
16033,ucutmH2KvSQ,28,Science & Technology,2020-09-02,2020-09-07,4222370,214557,3575,34939,False,False,7 days,2936406,101263,15421
481,CG5del3ZYts,22,People & Blogs,2020-09-04,2020-09-08,466275,53337,158,3574,False,False,4 days,249632,20981,1122
20272,czq7usfGiZY,17,Sports,2020-10-12,2020-10-15,2917433,40514,1801,9679,False,False,5 days,715878,5274,1035
1982,y4SQtg0G-2Y,20,Gaming,2020-08-19,2020-08-20,6357366,373460,6319,32063,False,False,6 days,4750099,110378,5742
9678,pvPsJFRGleA,10,Music,2020-09-18,2020-09-26,36350151,2331823,56544,175811,False,False,30 days,51246468,1795086,103090


### Count the difference in days between publish date and trending date 

In [19]:
# get the publish date and trending date 
published = total['publishedAt'].to_list()
trendy = total['trending_date'].to_list()

# count the difference 
total['diff_days'] = [x1 - x2 for (x1, x2) in zip(trendy, published)]

In [20]:
# look at the sample 
total.sample(10)

,video_id,category_id,category_title,publishedAt,trending_date,view_count,likes,dislikes,comment_count,comments_disabled,ratings_disabled,trending_days,view_increase,like_increase,comment_increase,diff_days
9837,VHrMlzTcAjs,10,Music,2020-09-18,2020-09-21,8680464,1013890,60046,75560,False,False,4 days,6234104,246201,29758,3 days
4924,BQWWIJJN6aY,24,Entertainment,2020-09-01,2020-09-03,361484,16537,539,2079,False,False,3 days,70414,1588,92,2 days
4388,G1mPnQM5hQ0,24,Entertainment,2020-08-21,2020-08-26,18831850,1072958,42186,86758,False,False,6 days,12157050,444014,28429,5 days
9581,EeaOTDREskE,10,Music,2020-09-14,2020-09-17,592051,18515,586,3581,False,False,5 days,487014,11105,2062,3 days
9929,g4P0HN5MPk4,10,Music,2020-09-21,2020-09-26,2790388,156675,5621,9032,False,False,5 days,2027785,77036,2609,5 days
25,gPdUslndvVI,22,People & Blogs,2020-08-11,2020-08-13,383578,42889,360,4148,False,False,5 days,520534,16056,1091,2 days
2263,Mcwyz09Q--o,20,Gaming,2020-09-01,2020-09-03,176456,9118,485,1021,False,False,3 days,17962,319,47,2 days
7669,Tc2MUxr6uVs,24,Entertainment,2020-11-18,2020-11-19,210781,12834,332,726,False,False,5 days,191743,7548,268,1 days
5632,ElVShISbYS4,24,Entertainment,2020-09-25,2020-09-26,514065,40605,355,2114,False,False,5 days,1378243,53594,1992,1 days
5262,UTpZxLM8GBg,24,Entertainment,2020-09-15,2020-09-19,1491422,28673,3082,3921,False,False,6 days,1361939,21779,3392,4 days


### Drop the duplicates

In [21]:
total = total.drop_duplicates('video_id')

### Create viewer engagement variable 

In [32]:
total['engagement'] = total['view_count'] + total['likes'] + total['comment_count'] + total['view_increase'] + total['like_increase'] + total['comment_increase']

In [35]:
# look at the sample
total.sort_values('engagement')

,video_id,category_id,category_title,publishedAt,trending_date,view_count,likes,dislikes,comment_count,comments_disabled,ratings_disabled,trending_days,view_increase,like_increase,comment_increase,diff_days,engagement
12364,AWXvClaRtsI,10,Music,2020-11-14,2020-11-15,0,0,0,0,True,True,6 days,0,0,0,1 days,0
6800,r7nYQXsxJdU,24,Entertainment,2020-10-25,2020-10-27,0,4257,384,240,False,False,2 days,0,163,21,2 days,4681
16580,ff1GJHq5W2o,28,Science & Technology,2020-10-13,2020-10-15,38510,1249,99,326,False,False,3 days,4962,146,41,2 days,45234
18945,tjXM6u36O98,17,Sports,2020-08-16,2020-08-17,47709,393,47,618,False,False,1 days,1027,3,19,1 days,49769
8353,vOpySfXMe6w,10,Music,2020-08-16,2020-08-17,44693,727,8,28,False,False,2 days,11754,28,4,1 days,57234
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8033,hsm4poTWjMs,10,Music,2020-08-07,2020-08-12,76805026,2820345,382578,270263,False,False,4 days,21637388,387384,40367,5 days,101960773
10417,dyRsYk0LyA8,10,Music,2020-10-02,2020-10-02,39363127,4746270,17513,1021283,False,False,7 days,101322312,4471606,486322,0 days,151410920
12474,-5q5mZbe3V8,10,Music,2020-11-20,2020-11-20,37570579,5177871,12125,2276126,False,False,8 days,113052202,6227159,1884777,0 days,166188714
8776,vRXZj0DzXIA,10,Music,2020-08-28,2020-08-28,51234434,5912778,405329,1847794,False,False,8 days,133543814,5882892,888203,0 days,199309915


### Export the dataset 

In [36]:
total.to_csv('trending_youtube.csv', index = False)